In [10]:
import uuid
from datetime import datetime, timezone

import boto3
from boto3.dynamodb.conditions import Key

In [18]:
project_name = "draft-declaration-rag-pipeline"
environment = "playground"

dynamodb_table_name = f"{project_name}-{environment}-dynamodb-table"

dynamodb = boto3.resource("dynamodb")
table = dynamodb.Table(dynamodb_table_name)

In [20]:
s3_path = f"s3://{project_name}-{environment}/data/input/client=clientA/psychology_notes.pdf"

dynamodb = boto3.resource("dynamodb")
item = {
    "id": str(uuid.uuid4()),
    "status": "PENDING",
    "input_path": s3_path,
    "created_at": datetime.now(timezone.utc).isoformat() + "Z",
}

# Insert the item
table.put_item(Item=item)
print("Item inserted successfully!")

Item inserted successfully!


In [10]:
# Query using the GSI 'status-index'
response = table.query(
    IndexName="status-index",
    KeyConditionExpression=Key("status").eq("PENDING"),
)

items = response.get("Items", [])
print(items)

item_ids = []

if not items:
    print("No PENDING items found.")
else:
    for item in items:
        item_ids.append(item["id"])

    for item_id in item_ids:
        # Step 2: Update status to FINISHED
        table.update_item(
            Key={"id": item_id},
            UpdateExpression="SET #s = :new_status",
            ExpressionAttributeNames={"#s": "status"},
            ExpressionAttributeValues={":new_status": "FINISHED"},
        )

    print(f"Item {item_id} status updated to FINISHED.")

[{'created_at': '2025-06-12T12:45:55.035739+00:00Z', 'id': 'eefd7b74-4b13-4c40-8f41-494ef07c061d', 'status': 'PENDING', 's3_path': 's3://your-bucket/path/to/file.txt'}]
Item eefd7b74-4b13-4c40-8f41-494ef07c061d status updated to FINISHED.
